In [ ]:
# default_exp model

# Models

> API details.

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
%matplotlib inline

In [ ]:
#export
import warnings
import re

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

import pytorch_lightning as pl
from pytorch_lightning.core import LightningModule
from pytorch_lightning.metrics import functional as FM

In [ ]:
#export
from isic.dataset import SkinDataModule
from isic.layers import LabelSmoothingCrossEntropy, LinBnDrop, AdaptiveConcatPool2d
from isic.callback.hyperlogger import HyperparamsLogger
from isic.callback.logtable import LogTableMetricsCallback
from isic.callback.mixup import MixupDict
from isic.callback.cutmix import CutmixDict
from isic.callback.freeze import FreezeCallback, UnfreezeCallback
from isic.utils import reduce_loss, apply_init, get_bias_batchnorm_params, apply_leaf, print_grad_module, generate_val_steps

In [ ]:
#export
def create_head(n_in, n_out, lin_ftrs=None, p=0.5, concat_pool=True):
    n_in = n_in * (2 if concat_pool else 1)
    lin_ftrs = [n_in, 512, n_out] if lin_ftrs is None else [n_in] + lin_ftrs + [n_out]
    p_dropouts = [p/2] * (len(lin_ftrs) - 2) + [p]
    activations = [nn.ReLU(inplace=True)] * (len(lin_ftrs) - 2) + [None]
    pool = AdaptiveConcatPool2d() if concat_pool else nn.AdaptiveAvgPool2d(1)
    layers = [pool, nn.Flatten()]
    for ni, no, p, actn in zip(lin_ftrs[:-1], lin_ftrs[1:], p_dropouts, activations):
        layers += LinBnDrop(ni, no, bn=True, p=p, act=actn)

    return nn.Sequential(*layers)

In [ ]:
#export
def params(m):
    "Return all parameters of `m`"
    return list(m.parameters())

def has_pool_type(m):
    def _is_pool_type(l): return re.search(r'Pool[123]d$', l.__class__.__name__)
    "Return `True` if `m` is a pooling layer or has one in its children"
    if _is_pool_type(m): return True
    for l in m.children():
        if has_pool_type(l): return True
    return False

def create_body(arch):
    def _xresnet_split(m):
        return [params(m[0][:3]), params(m[0][3:]), params(m[1:])]
    def _resnet_split(m):
        return [params(m[0][:6]), params(m[0][6:]), params(m[1:])]
    def _squeezenet_split(m):
        return [params(m[0][0][:5]), params(m[0][0][5:]), params(m[1:])]
    def _densenet_split(m:nn.Module): 
        return [params(m[0][0][:7]), params(m[0][0][7:]), params(m[1:])]
    def _vgg_split(m:nn.Module): 
        return [params(m[0][0][:22]), params(m[0][0][22:]), params(m[1:])]
    def _alexnet_split(m:nn.Module): 
        return [params(m[0][0][:6]), params(m[0][0][6:]), params(m[1:])]

    model = getattr(models, arch)(pretrained=True)
    num_ftrs = model.fc.in_features
    if 'xresnet' in arch:
        cut = -4
        split = _xresnet_split
    elif 'resnet':
        cut = -2
        split = _resnet_split
    elif 'squeeze':
        cut = -1
        split = _squeezenet_split
    elif 'dense':
        cut = -1
        split = _densenet_split
    elif 'vgg':
        cut = -2
        split = _vgg_split
    elif 'alex':
        cut = -2
        split = _alexnet_split
    else:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
        split = params
    return nn.Sequential(*list(model.children())[:cut]), split, num_ftrs

In [ ]:
#export
class Model(LightningModule):
    def __init__(self, steps_epoch, epochs=30, lr=1e-2, wd=0., n_out=7, concat_pool=True, arch='resnet50'):
        super().__init__()
        self.save_hyperparameters()
        # create body
        body, self.split, num_ftrs = create_body(arch)
        # create head
        head = create_head(num_ftrs, n_out)
        
        #model
        self.model = nn.Sequential(body, head)
        apply_init(self.model[1])
        
        self.loss_func = nn.CrossEntropyLoss()

    def get_params(self):
        return self.split(self.model)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch['img'], batch['label']
        y_hat = self(x)
        loss = self.loss_func(y_hat, y)
        acc = FM.accuracy(y_hat, y, num_classes=7)
        result = pl.TrainResult(minimize=loss)
        result.log('train_loss', loss)
        result.log('train_acc', acc, prog_bar=True)
        return result

    def validation_step(self, batch, batch_idx):
        x, y = batch['img'], batch['label']
        y_hat = self(x)
        loss = self.loss_func(y_hat, y)
        acc = FM.accuracy(y_hat, y, num_classes=7)
        result = pl.EvalResult(checkpoint_on=loss)
        result.log('val_loss', loss, prog_bar=True) 
        result.log('val_acc', acc, prog_bar=True)
        return result
    
    def create_opt(self, lr):
        def _inner():
            print('override_called')
            param_groups = self.get_params()
            n_groups = len(param_groups)
            lrs = generate_val_steps(lr, n_groups)
            assert len(lrs) == n_groups, f"Trying to set {len(lrs)} values for LR but there are {n_groups} parameter groups."
            grps = []
            for i in range(n_groups):
                grps.append({
                    "params": param_groups[i],
                    "lr": lrs[i]
                })
            print(lrs)
            opt = torch.optim.Adam(grps, 
                        lr=1e-2, weight_decay=self.hparams.wd
            )
            scheduler = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=lrs, steps_per_epoch=self.hparams.steps_epoch, epochs=self.hparams.epochs)
            sched = {
                'scheduler': scheduler, # The LR schduler
                'interval': 'step', # The unit of the scheduler's step size
                'frequency': 1, # The frequency of the scheduler
                'reduce_on_plateau': False, # For ReduceLROnPlateau scheduler
            }
            opt.t_state={}
            for p in get_bias_batchnorm_params(self.model):
                opt.t_state[p] = {"force_train": True}
            return [opt], [sched]
        self.configure_optimizers = _inner

In [ ]:
message_formater = "You have set {0} number of classes if different from predicted {0} and target {0} number of classes"
warnings.filterwarnings("ignore", message_formater.format("(.*)"), category=UserWarning)

In [ ]:
dm = SkinDataModule()
dm.prepare_data()
dm.setup('fit')

In [ ]:
EPOCHS = 10
STEPS_EPOCH = 1

In [ ]:
# init model
model = Model(steps_epoch=STEPS_EPOCH, epochs=EPOCHS, lr=1e-2)
model.create_opt(slice(1e-3, 1))

In [ ]:
# Freeze training
hp_log = HyperparamsLogger()
freeze_cb = FreezeCallback()

trainer = pl.Trainer(max_epochs=EPOCHS, callbacks=[LogTableMetricsCallback(), hp_log, freeze_cb], fast_dev_run=True, limit_val_batches=0, limit_train_batches=0.01)

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, dm)

C:\Users\bomco\miniconda3\envs\devtorch\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 25 M  
1 | loss_func | CrossEntropyLoss | 0     


override_called
[0.001, 0.03162277660168379, 1.0]
wtf


epoch,train_loss,train_acc,val_loss,val_acc
1,3.252,0.156250,78.051300,0.000000


Saving latest checkpoint..


1

In [ ]:
trainer.optimizers[0]

Adam (
Parameter Group 0
    amsgrad: False
    base_momentum: 0.85
    betas: (0.8999999999999999, 0.999)
    eps: 1e-08
    initial_lr: 4e-05
    lr: 0.0005200000000000001
    max_lr: 0.001
    max_momentum: 0.95
    min_lr: 4e-09
    weight_decay: 0.0

Parameter Group 1
    amsgrad: False
    base_momentum: 0.85
    betas: (0.8999999999999999, 0.999)
    eps: 1e-08
    initial_lr: 0.0012649110640673518
    lr: 0.016443843832875574
    max_lr: 0.03162277660168379
    max_momentum: 0.95
    min_lr: 1.2649110640673517e-07
    weight_decay: 0.0

Parameter Group 2
    amsgrad: False
    base_momentum: 0.85
    betas: (0.8999999999999999, 0.999)
    eps: 1e-08
    initial_lr: 0.04
    lr: 0.52
    max_lr: 1.0
    max_momentum: 0.95
    min_lr: 4e-06
    weight_decay: 0.0
)

In [ ]:
# Unfreeze training
hp_log = HyperparamsLogger()
freeze_cb = FreezeCallback()

trainer = pl.Trainer(max_epochs=EPOCHS, callbacks=[LogTableMetricsCallback(), hp_log, unfreeze_cb], fast_dev_run=True, limit_val_batches=0, limit_train_batches=0.01)
model.create_opt(1e-10)

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, dm)


  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 25 M  
1 | loss_func | CrossEntropyLoss | 0     


override_called
wtf


epoch,train_loss,train_acc,val_loss,val_acc
1,3.256,0.156250,4.279556,0.000000


Saving latest checkpoint..


1

In [ ]:
print_grad_module(model.model[1])

AdaptiveAvgPool2d(output_size=1)
[]
AdaptiveMaxPool2d(output_size=1)
[]
Flatten()
[]
BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
[True, True]
Dropout(p=0.25, inplace=False)
[]
Linear(in_features=4096, out_features=512, bias=False)
[True]
ReLU(inplace=True)
[]
BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
[True, True]
Dropout(p=0.5, inplace=False)
[]
Linear(in_features=512, out_features=7, bias=False)
[True]


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=lightning_logs/

Reusing TensorBoard on port 6006 (pid 4636), started 4 days, 18:33:26 ago. (Use '!kill 4636' to kill it.)

In [ ]:
!tensorboard --logdir=lightning_logs/

In [ ]:
# #
# class ResnetModel(LightningModule):
#     def __init__(self):
#         super().__init__()
# #         self.save_hyperparameters()
#         self.resnet = models.resnet50(pretrained=True)
#         num_ftrs = self.resnet.fc.in_features
#         self.resnet.fc = nn.Linear(num_ftrs, 7)
#         self.loss_func = F.cross_entropy

#     def forward(self, x):
#         return self.resnet(x)

#     def training_step(self, batch, batch_idx):
#         print(batch)
#         if hasattr(self, 'enhanced_batch'):
#             print('hehe')
#             batch = self.enhanced_batch
#         x, y = batch
#         y_hat = self(x)
#         loss = self.loss_func(y_hat, y)
#         acc = FM.accuracy(y_hat, y, num_classes=7)
#         result = pl.TrainResult(minimize=loss)
#         result.log('train_loss', loss)
#         result.log('train_acc', acc, prog_bar=True)
#         return result

#     def validation_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = F.cross_entropy(y_hat, y)
#         acc = FM.accuracy(y_hat, y, num_classes=7)
#         result = pl.EvalResult(checkpoint_on=loss)
#         result.log('val_loss', loss, prog_bar=True) 
#         result.log('val_acc', acc, prog_bar=True)
#         return result

#     def configure_optimizers(self):
#         opt = torch.optim.Adam(self.parameters(), lr=1e-2)
#         scheduler = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=1e-2, steps_per_epoch=140, epochs=10)
#         sched = {
#             'scheduler': scheduler, # The LR schduler
#             'interval': 'step', # The unit of the scheduler's step size
#             'frequency': 1, # The frequency of the scheduler
#             'reduce_on_plateau': False, # For ReduceLROnPlateau scheduler
#         }
#         return [opt], [sched]

In [ ]:
from nbdev.export import *
notebook2script('model.ipynb')

Converted cb_mixup.ipynb.
